<a href="https://colab.research.google.com/github/Aaron-David-HdM/Applied-Data-Analytics/blob/main/UseCase_1/Datenimport_und_Bereinigung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Zeigt Inhalte des gewählten Ordners**

In [ ]:
ls

Applied-Data-Analytics/  sample_data/


**Verknüpfung zum GitHub-Repository**

In [2]:
!git clone https://github.com/Aaron-David-HdM/Applied-Data-Analytics.git

Cloning into 'Applied-Data-Analytics'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 139 (delta 47), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (139/139), 33.40 KiB | 621.00 KiB/s, done.
Resolving deltas: 100% (47/47), done.


**Wechselt zum lokalen Repository-Verzeichnis und holt Daten vom GitHub-Repo**

In [24]:
%cd /content/Applied-Data-Analytics/
!git pull

/content/Applied-Data-Analytics
Already up to date.


**Bibliotheken-Import, Wechsel zum lokalen Repository und lädt die CSV-Dateien.**

In [25]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

%cd /content/Applied-Data-Analytics/UseCase_1

filepath_1 = 'Rohdaten_18-34_Jahre.csv' # Gesamtzahl der Nutzer,18-24 sowie 25-34 jährige Nutzer
filepath_2 = 'Rohdaten_35-65+_Jahre.csv' # 35-44,	45-54 und 55-64 jährige Nutzer sowie Nutzer über 65 Jahren


df_1 = pd.read_csv(filepath_1, decimal=',')
df_2 = pd.read_csv(filepath_2, decimal=',')

/content/Applied-Data-Analytics/UseCase_1


**Daten Bereinigung**

Google Analytics erlaubt nur die Verknüpfung von bis zu 4 verschiedene Segmenten bei der Erzeugung der Daten. Somit waren wir gezwungen, zwei Tabellen anzulegen und diese im Anschluss miteinander zu 'mergen'.


**Löschen von NaN-Werten und der 'Zeitraum'-Column sowie dem Ersetzen der Punkte bei den Zahlenwerten**

In [26]:
new_df_1 = df_1.dropna()
new_df_1 = df_1.drop(columns=['Zeitraum'])
new_df_1['Nutzer'] = new_df_1['Nutzer'].str.replace('.', '')

new_df_2 = df_2.dropna()
new_df_2 = df_2.drop(columns=['Zeitraum'])
new_df_2['Nutzer'] = new_df_2['Nutzer'].str.replace('.', '')

**Columns umbenannt und neu gruppiert.**

In [27]:
new_df_1 = new_df_1.rename(columns={'Segment':'Altersgruppe', 'Index: Tag':'Tag'})
new_df_1 = new_df_1.groupby(['Tag', 'Altersgruppe'])['Nutzer'].first().unstack()
new_df_1.columns = ['Alle Nutzer', '18-24', '25-34']

new_df_2 = new_df_2.rename(columns={'Segment':'Altersgruppe', 'Index: Tag':'Tag'})
new_df_2 = new_df_2.groupby(['Tag', 'Altersgruppe'])['Nutzer'].first().unstack()
new_df_2.columns = ['35-44', '45-54', '55-64', '65+']

**Nutzerzahlen zu Integern konvertiert.**

In [28]:
new_df_1[['18-24', '25-34', 'Alle Nutzer']] = new_df_1[['18-24', '25-34', 'Alle Nutzer']].apply(np.int64)

new_df_2[['35-44', '45-54', '55-64', '65+']] = new_df_2[['35-44', '45-54', '55-64', '65+']].apply(np.int64)

**Die zwei Dataframes zu einem Dataframe zusammengeführt.**

In [29]:
df = new_df_1.merge(new_df_2, how='inner', left_index=True, right_index=True)

**Nutzerzahlen je Tag addieren und Werte zum Datenframe hinzufügen**

In [30]:
df['Alle Nutzer_2'] = df.iloc[:, 1:].sum(axis=1)

**Dataframe als CSV-Datei abspeichern**

In [31]:
df.to_csv('Testgruppe_A', sep=',')